<a href="https://colab.research.google.com/github/grace3999/USV_Python/blob/colab/0_xr_create_annotations_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive containings required files: 1) csv of annotation features, 2) netcdf files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [0]:
def get_dir_list(dir_path):
    """takes in a directory path which contains the selection tables of annotations created from Raven Soft Lite, 
    creates list of path names"""
    
    path_names = []
    dirs = os.listdir(dir_path)
    for file in dirs: 
        path_names.append(dir_path + "/" + file)

    return path_names   

In [0]:
#may need to be updated based on file naming scheme
def get_file_info(file_path_name):
    """takes in a file path and finds the animal_number and session and saves each accordingly. 
    each file should be named with animal number and exp (e.g. 100_CPA.Table.1.selections)"""
    
    animal_number, session = file_path_name.split('/')[-1].split('-')[1], file_path_name.split('/')[-1].split('-')[0]
    print(animal_number, session)

    return animal_number, session

In [0]:
def create_animal_df(file_path_name, animal_number, session):
    """takes in a file path, animal number and session and creates data frame.
    adds animal number, session, and time stamp columns"""
    
    data = pd.read_table(file_path_name)
    data = pd.DataFrame(data = data)
    
    data['animal_number'] = [animal_number]*data.shape[0]
    data['session'] = [session]*data.shape[0]
    
    #Determine closest time stamp of each annotation and add as column to df
    data['Begin Time (s)_1000'] = data['Begin Time (s)']*1000
    time_lambda = lambda a: (a // 22.5)*22.5
    data['time_stamp'] = time_lambda(data['Begin Time (s)_1000'])
    
    data = data[['animal_number', 'session', 'time_stamp', 'Annotation', 'High Freq (Hz)']]

    return data

In [0]:
def create_annot_df(dir_path, session_name):
    """takes in a directory path which contains the selection tables of annotations created from Raven Soft Lite,
    uses get_dir_list, get_file_info, and create_animal_df functions 
    to create a single data frame containing annotations for each file in dir_path"""
    
    annot_df = pd.DataFrame()
    
    path_names = get_dir_list(dir_path)
    
    for path in path_names:
        animal_number, session = get_file_info(path)
        animal_df = create_animal_df(path, animal_number, session)
        annot_df = annot_df.append(animal_df)
    
    #save as csv using name of dir_path
    annot_df.to_csv(str('annot_df_' + str(session_name) + '.csv'))
    
    return annot_df

In [33]:
annot_df = create_annot_df(dir_path, 'pain')
print(annot_df.shape)
annot_df.head()

A2 D2
A6 D2
A8 D3
A4 D3
A10 D2
A6 D3
A12 D2
A4 D2
A2 D3
A8 D2
A10 D3
A12 D4
A2 D4
A8 D4
A10 D4
A4 D4
A12 D3
A6 D4
(178, 5)


,animal_number,session,time_stamp,Annotation,High Freq (Hz)
0,A2,D2,32692.5,NaN,48403.2
1,A2,D2,97515.0,NaN,50048.4
2,A2,D2,124942.5,NaN,41465.2
3,A2,D2,292027.5,NaN,48718.6
4,A2,D2,296505.0,NaN,52345.3
